# 附录A NumPy高级应用

在这篇附录中，我会深入NumPy库的数组计算。这会包括ndarray更内部的细节，和更高级的数组操作和算法。

本章包括了一些杂乱的章节，不需要仔细研究。

# A.1 ndarray对象的内部机理

NumPy的ndarray提供了一种将同质数据块（可以是连续或跨越）解释为多维数组对象的方式。正如你之前所看到的那样，数据类型（dtype）决定了数据的解释方式，比如浮点数、整数、布尔值等。

ndarray如此强大的部分原因是所有数组对象都是数据块的一个跨度视图（strided view）。你可能想知道数组视图arr[::2,::-1]不复制任何数据的原因是什么。简单地说，ndarray不只是一块内存和一个dtype，它还有跨度信息，这使得数组能以各种步幅（step size）在内存中移动。更准确地讲，ndarray内部由以下内容组成：

- 一个指向数据（内存或内存映射文件中的一块数据）的指针。
- 数据类型或dtype，描述在数组中的固定大小值的格子。
- 一个表示数组形状（shape）的元组。
- 一个跨度元组（stride），其中的整数指的是为了前进到当前维度下一个元素需要“跨过”的字节数。

图A-1简单地说明了ndarray的内部结构。

例如，一个10×5的数组，其形状为(10,5)：

In [4]:
import numpy as np

In [5]:
np.ones((10, 5)).shape

(10, 5)

一个典型的（C顺序，稍后将详细讲解）3×4×5的float64（8个字节）数组，其跨度为(160,40,8) —— 知道跨度是非常有用的，通常，跨度在一个轴上越大，沿这个轴进行计算的开销就越大：

In [6]:
np.ones((3, 4, 5), dtype=np.float64).strides

(160, 40, 8)

虽然NumPy用户很少会对数组的跨度信息感兴趣，但它们却是构建非复制式数组视图的重要因素。跨度甚至可以是负数，这样会使数组在内存中后向移动，比如在切片obj `[::-1]` 或obj `[:,::-1]` 中就是这样的。

## NumPy数据类型体系

你可能偶尔需要检查数组中所包含的是否是整数、浮点数、字符串或Python对象。因为浮点数的种类很多（从float16到float128），判断dtype是否属于某个大类的工作非常繁琐。幸运的是，dtype都有一个超类（比如np.integer和np.floating），它们可以跟np.issubdtype函数结合使用：

In [8]:
ints = np.ones(10, dtype=np.uint16)

floats = np.ones(10, dtype=np.float32)

np.issubdtype(ints.dtype, np.integer)

True

In [9]:
np.issubdtype(floats.dtype, np.floating)

True

调用dtype的mro方法即可查看其所有的父类：

In [10]:
np.float64.mro()

[numpy.float64,
 numpy.floating,
 numpy.inexact,
 numpy.number,
 numpy.generic,
 float,
 object]

然后得到：

In [11]:
np.issubdtype(ints.dtype, np.number)

True

大部分NumPy用户完全不需要了解这些知识，但是这些知识偶尔还是能派上用场的。图A-2说明了dtype体系以及父子类关系。

# A.2 高级数组操作

除花式索引、切片、布尔条件取子集等操作之外，数组的操作方式还有很多。虽然pandas中的高级函数可以处理数据分析工作中的许多重型任务，但有时你还是需要编写一些在现有库中找不到的数据算法。

## 数组重塑

多数情况下，你可以无需复制任何数据，就将数组从一个形状转换为另一个形状。只需向数组的实例方法reshape传入一个表示新形状的元组即可实现该目的。例如，假设有一个一维数组，我们希望将其重新排列为一个矩阵（结果见图A-3）：

In [12]:
arr = np.arange(8)

arr

array([0, 1, 2, 3, 4, 5, 6, 7])

In [13]:
arr.reshape((4, 2))

array([[0, 1],
       [2, 3],
       [4, 5],
       [6, 7]])

多维数组也能被重塑：

In [14]:
arr.reshape((4, 2)).reshape((2, 4))

array([[0, 1, 2, 3],
       [4, 5, 6, 7]])

作为参数的形状的其中一维可以是－1，它表示该维度的大小由数据本身推断而来：

In [15]:
arr = np.arange(15)

arr.reshape((5, -1))

array([[ 0,  1,  2],
       [ 3,  4,  5],
       [ 6,  7,  8],
       [ 9, 10, 11],
       [12, 13, 14]])

与reshape将一维数组转换为多维数组的运算过程相反的运算通常称为扁平化（flattening）或散开（raveling）：

In [16]:
arr = np.arange(15).reshape((5, 3))

arr

array([[ 0,  1,  2],
       [ 3,  4,  5],
       [ 6,  7,  8],
       [ 9, 10, 11],
       [12, 13, 14]])

In [17]:
arr.ravel()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14])

如果结果中的值与原始数组相同，ravel不会产生源数据的副本。flatten方法的行为类似于ravel，只不过它总是返回数据的副本：

In [18]:
arr.flatten()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14])

数组可以被重塑或散开为别的顺序。这对NumPy新手来说是一个比较微妙的问题，所以在下一小节中我们将专门讲解这个问题。

## C和Fortran顺序

NumPy允许你更为灵活地控制数据在内存中的布局。默认情况下，NumPy数组是按行优先顺序创建的。在空间方面，这就意味着，对于一个二维数组，每行中的数据项是被存放在相邻内存位置上的。另一种顺序是列优先顺序，它意味着每列中的数据项是被存放在相邻内存位置上的。

由于一些历史原因，行和列优先顺序又分别称为C和Fortran顺序。在FORTRAN 77中，矩阵全都是列优先的。

像reshape和reval这样的函数，都可以接受一个表示数组数据存放顺序的order参数。一般可以是'C'或'F'（还有'A'和'K'等不常用的选项，具体请参考NumPy的文档）。图A-3对此进行了说明：

In [19]:
arr = np.arange(12).reshape((3, 4))

arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [20]:
arr.ravel()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])

In [21]:
arr.ravel('F')

array([ 0,  4,  8,  1,  5,  9,  2,  6, 10,  3,  7, 11])

二维或更高维数组的重塑过程比较令人费解（见图A-3）。C和Fortran顺序的关键区别就是维度的行进顺序：

- C/行优先顺序：先经过更高的维度（例如，轴1会先于轴0被处理）。
- Fortran/列优先顺序：后经过更高的维度（例如，轴0会先于轴1被处理）。


## 数组的合并和拆分

numpy.concatenate可以按指定轴将一个由数组组成的序列（如元组、列表等）连接到一起：

In [22]:
arr1 = np.array([[1, 2, 3], [4, 5, 6]])

arr2 = np.array([[7, 8, 9], [10, 11, 12]])

np.concatenate([arr1, arr2], axis=0)

array([[ 1,  2,  3],
       [ 4,  5,  6],
       [ 7,  8,  9],
       [10, 11, 12]])

In [23]:
np.concatenate([arr1, arr2], axis=1)

array([[ 1,  2,  3,  7,  8,  9],
       [ 4,  5,  6, 10, 11, 12]])

对于常见的连接操作，NumPy提供了一些比较方便的方法（如vstack和hstack）。因此，上面的运算还可以表达为：

In [24]:
np.vstack((arr1, arr2))

array([[ 1,  2,  3],
       [ 4,  5,  6],
       [ 7,  8,  9],
       [10, 11, 12]])

In [25]:
np.hstack((arr1, arr2))

array([[ 1,  2,  3,  7,  8,  9],
       [ 4,  5,  6, 10, 11, 12]])

与此相反，split用于将一个数组沿指定轴拆分为多个数组：

In [26]:
arr = np.random.randn(5, 2)

arr

array([[-0.40014685,  0.57754405],
       [-1.15294229,  0.39250443],
       [ 0.07094194, -1.95466785],
       [-1.01733544,  0.83630226],
       [ 0.84928065, -0.71610903]])

In [27]:
first, second, third = np.split(arr, [1, 3])

first

array([[-0.40014685,  0.57754405]])

In [28]:
second

array([[-1.15294229,  0.39250443],
       [ 0.07094194, -1.95466785]])

In [29]:
third

array([[-1.01733544,  0.83630226],
       [ 0.84928065, -0.71610903]])

传入到np.split的值 [1,3] 指示在哪个索引处分割数组。

表A-1中列出了所有关于数组连接和拆分的函数，其中有些是专门为了方便常见的连接运算而提供的。

## 堆叠辅助类：`r_`和`c_`

NumPy命名空间中有两个特殊的对象—— `r_` 和 `c_`，它们可以使数组的堆叠操作更为简洁：

In [30]:
arr = np.arange(6)


arr1 = arr.reshape((3, 2))
arr2 = np.random.randn(3, 2)


np.r_[arr1, arr2]

array([[ 0.        ,  1.        ],
       [ 2.        ,  3.        ],
       [ 4.        ,  5.        ],
       [-0.06618184, -0.73842699],
       [ 0.44299995,  0.79669088],
       [-0.90790227, -1.02390444]])

In [31]:
np.c_[np.r_[arr1, arr2], arr]

array([[ 0.        ,  1.        ,  0.        ],
       [ 2.        ,  3.        ,  1.        ],
       [ 4.        ,  5.        ,  2.        ],
       [-0.06618184, -0.73842699,  3.        ],
       [ 0.44299995,  0.79669088,  4.        ],
       [-0.90790227, -1.02390444,  5.        ]])

它还可以将切片转换成数组：

In [32]:
np.c_[1:6, -10:-5]

array([[  1, -10],
       [  2,  -9],
       [  3,  -8],
       [  4,  -7],
       [  5,  -6]])

`r_` 和 `c_` 的具体功能请参考其文档。

## 元素的重复操作：tile和repeat

对数组进行重复以产生更大数组的工具主要是repeat和tile这两个函数。repeat会将数组中的各个元素重复一定次数，从而产生一个更大的数组：

In [34]:
arr = np.arange(3)

arr

array([0, 1, 2])

In [35]:
arr.repeat(3)

array([0, 0, 0, 1, 1, 1, 2, 2, 2])

> 笔记：跟其他流行的数组编程语言（如MATLAB）不同，NumPy中很少需要对数组进行重复（replicate）。这主要是因为广播（broadcasting，我们将在下一节中讲解该技术）能更好地满足该需求。

默认情况下，如果传入的是一个整数，则各元素就都会重复那么多次。如果传入的是一组整数，则各元素就可以重复不同的次数：

In [36]:
arr.repeat([2, 3, 4])

array([0, 0, 1, 1, 1, 2, 2, 2, 2])

对于多维数组，还可以让它们的元素沿指定轴重复：

In [38]:
arr = np.random.randn(2, 2)

arr

array([[-1.65925639, -0.32191027],
       [-0.50620784, -0.18262158]])

In [39]:
arr.repeat(2, axis=0)

array([[-1.65925639, -0.32191027],
       [-1.65925639, -0.32191027],
       [-0.50620784, -0.18262158],
       [-0.50620784, -0.18262158]])

注意，如果没有设置轴向，则数组会被扁平化，这可能不会是你想要的结果。同样，在对多维进行重复时，也可以传入一组整数，这样就会使各切片重复不同的次数：

In [40]:
arr.repeat([2, 3], axis=0)

array([[-1.65925639, -0.32191027],
       [-1.65925639, -0.32191027],
       [-0.50620784, -0.18262158],
       [-0.50620784, -0.18262158],
       [-0.50620784, -0.18262158]])

In [41]:
arr.repeat([2, 3], axis=1)

array([[-1.65925639, -1.65925639, -0.32191027, -0.32191027, -0.32191027],
       [-0.50620784, -0.50620784, -0.18262158, -0.18262158, -0.18262158]])

tile 的功能是沿指定轴向堆叠数组的副本。你可以形象地将其想象成“铺瓷砖”：

In [42]:
arr

array([[-1.65925639, -0.32191027],
       [-0.50620784, -0.18262158]])

In [43]:
np.tile(arr, 2)

array([[-1.65925639, -0.32191027, -1.65925639, -0.32191027],
       [-0.50620784, -0.18262158, -0.50620784, -0.18262158]])

第二个参数是瓷砖的数量。对于标量，瓷砖是水平铺设的，而不是垂直铺设。它可以是一个表示“铺设”布局的元组：

In [44]:
arr

array([[-1.65925639, -0.32191027],
       [-0.50620784, -0.18262158]])

In [45]:
np.tile(arr, (2, 1))

array([[-1.65925639, -0.32191027],
       [-0.50620784, -0.18262158],
       [-1.65925639, -0.32191027],
       [-0.50620784, -0.18262158]])

In [46]:
np.tile(arr, (3, 2))

array([[-1.65925639, -0.32191027, -1.65925639, -0.32191027],
       [-0.50620784, -0.18262158, -0.50620784, -0.18262158],
       [-1.65925639, -0.32191027, -1.65925639, -0.32191027],
       [-0.50620784, -0.18262158, -0.50620784, -0.18262158],
       [-1.65925639, -0.32191027, -1.65925639, -0.32191027],
       [-0.50620784, -0.18262158, -0.50620784, -0.18262158]])

## 花式索引的等价函数：take和put

在第4章中我们讲过，获取和设置数组子集的一个办法是通过整数数组使用花式索引：

In [47]:
arr = np.arange(10) * 100

inds = [7, 1, 2, 6]


arr[inds]

array([700, 100, 200, 600])

ndarray还有其它方法用于获取单个轴向上的选区：

In [48]:
arr.take(inds)


arr.put(inds, 42)


arr

array([  0,  42,  42, 300, 400, 500,  42,  42, 800, 900])

In [50]:
arr.put(inds, [40, 41, 42, 43])

arr

array([  0,  41,  42, 300, 400, 500,  43,  40, 800, 900])

要在其它轴上使用take，只需传入axis关键字即可：

In [51]:
inds = [2, 0, 2, 1]

arr = np.random.randn(2, 4)

arr

array([[-1.61566876, -0.19050563,  0.0156007 ,  0.13756615],
       [ 0.69576078,  0.78433202,  0.87687069,  0.32608304]])

In [52]:
arr.take(inds, axis=1)

array([[ 0.0156007 , -1.61566876,  0.0156007 , -0.19050563],
       [ 0.87687069,  0.69576078,  0.87687069,  0.78433202]])

put不接受axis参数，它只会在数组的扁平化版本（一维，C顺序）上进行索引。因此，在需要用其他轴向的索引设置元素时，最好还是使用花式索引。